## Semana 2 Día 2

¡Nuestro primer proyecto de Agentic Framework!

Prepárate para algo increíblemente fácil.

Vamos a crear un sistema de agentes sencillo para generar correos electrónicos de contacto de ventas en frío:
1. Flujo de trabajo del agente
2. Uso de herramientas para llamar a funciones
3. Colaboración de los agentes mediante herramientas y transferencias

## Antes de empezar, algunos pasos de configuración:

Visita Sendgrid en: https://sendgrid.com/

(Sendgrid es una empresa de Twilio para enviar correos electrónicos).

Crea una cuenta. ¡Es gratis! (al menos para mí, por ahora).

Una vez creada, haz clic en:

Configuración (barra lateral izquierda) >> Claves API >> Crear clave API (botón en la esquina superior derecha).

Copia la clave al portapapeles y añade una nueva línea a tu archivo .env:

`SENDGRID_API_KEY=xxxx`

Y también, dentro de SendGrid, ve a:

Configuración (barra lateral izquierda) >> Autenticación del remitente >> "Verificar un solo remitente"
y verifica que tu dirección de correo electrónico sea real para que SendGrid pueda enviar correos electrónicos por ti.

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio



In [ ]:
load_dotenv(override=True)

## Paso 1: Flujo de trabajo del agente

In [3]:
instructions1 = "Eres un agente de ventas que trabaja para ComplAI, \
    una empresa que ofrece una herramienta SaaS para garantizar el cumplimiento de SOC2\
          y prepararse para auditorías, impulsada por IA. Redactas correos electrónicos profesionales y serios."

instructions2 = "Eres un agente de ventas con sentido del humor y atractivo \
    que trabaja para ComplAI, una empresa que ofrece una herramienta SaaS para \
        garantizar el cumplimiento de SOC2 y prepararse para auditorías, impulsada por IA. \
            Redactas correos electrónicos ingeniosos y atractivos que probablemente obtengan respuesta."

instructions3 = "Eres un agente de ventas muy activo que trabaja para ComplAI, \
    una empresa que ofrece una herramienta SaaS para garantizar el cumplimiento de SOC2\
          y prepararse para auditorías, impulsada por IA. Redactas correos electrónicos concisos y directos."

In [4]:
sales_agent1 = Agent(
        name="Agente de ventas profesional",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Agente de ventas atractivo",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Agente de ventas ocpado",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [ ]:

result = Runner.run_streamed(sales_agent1, input="Escribe un correo electrónico de ventas en frío")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

In [ ]:
message = "Escribe un correo electrónico de ventas en frío"

with trace("Correos electrónicos fríos en paralelo"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


In [7]:
sales_picker = Agent(
    name="sales_picker",
    instructions="Elige el mejor correo electrónico de ventas en frío entre las opciones disponibles. \
            Imagina que eres un cliente y elige el que probablemente te responda. \
            No des explicaciones; responde solo con el correo electrónico seleccionado.",
    model="gpt-4o-mini"
)

In [ ]:
message = "Escribe un correo electrónico de ventas en frío"

with trace("Selección del equipo de ventas"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Emails de ventas en frío:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"El mejor email de ventas:\n{best.final_output}")


Vamos a revisar ahora la traza:

https://platform.openai.com/traces

## Parte 2: Uso de herramientas

Ahora añadiremos una herramienta.

Recuerda todo el código JSON repetitivo y la función `handle_tool_calls()` con la lógica `if`.

In [9]:
sales_agent1 = Agent(
        name="Agente de ventas profesional",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Agente de ventas atractivo",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Agente de ventas ocpado",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [ ]:
sales_agent1

## Pasos 2 y 3: Interacción con herramientas y agentes

¿Recuerdas todo ese JSON repetitivo?

Simplemente encapsula tu función con el decorador `@function_tool`

In [11]:
@function_tool
def send_email(body: str):
    """ Envía un correo electrónico con el cuerpo indicado a todos los clientes potenciales de ventas. """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("juangabriel@frogames.es")  # Cambiar a tu remitente verificado
    to_email = To("juangabriel@frogames.es")  # Cambiar a sureceptor
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Email de ventas", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### Esto se ha convertido automáticamente en una herramienta, con el json repetitivo creado

In [ ]:
# Echémosle un vistazo
send_email

### Y también puedes convertir un Agente en una herramienta

In [ ]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Escribe un email de ventas en frío")
tool1

### Ahora podemos reunir todas las herramientas:

Una herramienta para cada uno de nuestros tres agentes de redacción de correos electrónicos

Y una herramienta para nuestra función de envío de correos electrónicos

In [ ]:
description = "Escribe un correo electrónico de ventas en frío"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

## Y ahora es el momento para nuestro Gerente de Ventas - nuestro agente de planificación

In [15]:
instructions ="Eres gerente de ventas y trabajas para ComplAI. \
Utilizas las herramientas que te proporcionamos para generar correos electrónicos de ventas en frío. \
Nunca generas correos electrónicos de ventas tú mismo; siempre usas las herramientas. \
Pruebas las tres herramientas de sales_agent una vez antes de elegir la mejor. \
Eliges el mejor correo electrónico y usas la herramienta send_email para enviar el mejor correo electrónico (y solo el mejor) al usuario."


sales_manager = Agent(name="Manager de ventas", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Envíe un correo electrónico de ventas frío dirigido a 'Estimado director ejecutivo'"

with trace("Manager de ventas"):
    result = await Runner.run(sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Espera, ¿no recibiste un correo electrónico?</h2>
<span style="color:#ff7800;">Muchas gracias al estudiante Chris S. por describir su problema y las soluciones.

Si no recibes un correo electrónico después de ejecutar la celda anterior, revisa lo siguiente: <br/>

Primero, ¡revisa tu carpeta de correo no deseado! ¡Varios estudiantes no se han dado cuenta de que los correos electrónicos llegaron a esa carpeta! <br/>
Segundo, imprime (resultado) y comprueba si recibes errores sobre SSL.

Si recibes errores de SSL, consulta estos <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">consejos de red</a> y lee la nota en la siguiente celda. También revisa el seguimiento en OpenAI e investiga en el sitio web de SendGrid para encontrar pistas. ¡Avísame si puedo ayudarte!
        </td>
    </tr>
</table>

### Y una sugerencia más para enviar correos electrónicos del estudiante Oleksandr en Windows 11:

Si recibes errores de certificado SSL, entonces:
Ejecuta esto en una terminal: `uv pip install --upgrade certifi`

Luego ejecuta este código:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

¡Gracias, Oleksandr!

## Recuerda revisar el seguimiento

https://platform.openai.com/traces

¡Y luego revisa tu correo electrónico!

### Las transferencias (handoffs) representan una forma en que un agente puede delegar en otro agente, transfiriéndole el control.

Las transferencias y los agentes como herramientas son similares:

En ambos casos, un agente puede colaborar con otro.

Con las herramientas, el control se transfiere.

Con las transferencias, el control se transfiere.

In [16]:
subject_instructions = "Puedes escribir un asunto para un correo electrónico de ventas en frío. \
    Se te proporciona un mensaje y necesitas escribir un asunto para un correo electrónico que probablemente obtenga respuesta."

html_instructions = "Puedes convertir un cuerpo de correo electrónico de texto a un cuerpo de correo electrónico HTML. \
    Se te proporciona un cuerpo de correo electrónico de texto que puede tener algún markdown \
    y necesitas convertirlo a un cuerpo de correo electrónico HTML con un diseño simple, claro y atractivo."

subject_writer = Agent(name="Escritor de asunto de correo electrónico", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Escribe un asunto para un correo electrónico de ventas en frío")

html_converter = Agent(name="Conversor de cuerpo de correo electrónico HTML", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convierte un cuerpo de correo electrónico de texto a un cuerpo de correo electrónico HTML")


In [18]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Envía un correo electrónico con el asunto y el cuerpo HTML a todos los clientes potenciales de ventas """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("juangabriel@frogames.es")  # Cambiar a tu remitente verificado
    to_email = To("juangabriel@frogames.es")  # Cambiar a sureceptor
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [19]:
tools = [subject_tool, html_tool, send_html_email]

In [ ]:
tools

In [21]:
instructions ="Eres un formateador y remitente de correos electrónicos. \
    Recibes el cuerpo de un correo electrónico para enviarlo. \
    Primero usas la herramienta subject_writer para escribir un asunto para el correo electrónico, \
    luego usas la herramienta html_converter para convertir el cuerpo a HTML. \
    Finalmente, usas la herramienta send_html_email para enviar el correo electrónico con el asunto y el cuerpo HTML."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convierte un email a HTML y lo envía")


### Ahora tenemos 3 herramientas y 1 transferencia

In [ ]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

In [24]:
sales_manager_instructions = "Eres un gerente de ventas que trabaja para ComplAI. Utilizas las herramientas que se te proporcionan para generar correos electrónicos de ventas en frío. \
Nunca generas correos electrónicos de ventas tú mismo; siempre utilizas las herramientas. \
Pruebas las 3 herramientas del agente de ventas al menos una vez antes de elegir la mejor. \
Puedes usar las herramientas múltiples veces si no estás satisfecho con los resultados del primer intento. \
Seleccionas el mejor correo electrónico usando tu propio criterio sobre cuál será más efectivo. \
Después de elegir el correo electrónico, transfieres al agente Email Manager para formatear y enviar el correo."


sales_manager = Agent(
    name="Manager de ventas",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Envíe un correo electrónico de ventas en frío dirigido a 'Estimado director ejecutivo'"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

### Recuerda revisar el seguimiento

https://platform.openai.com/traces

¡Y luego revisa tu correo electrónico!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Ejercicio</h2>
<span style="color:#ff7800;">¿Puedes identificar los patrones de diseño de Agentic que se han usado aquí?<br/>
¿Cuál es la línea que cambió esto de ser un "flujo de trabajo" de Agentic a un "agente" según la definición de Anthropic?<br/>
¡Intenta agregar más herramientas y agentes! Podrías tener herramientas que gestionen la combinación de correspondencia para enviar a una lista.<br/><br/>
RETO DIFÍCIL: Investiga cómo puedes hacer que SendGrid llame a un webhook de devolución de llamada cuando un usuario responde a un correo electrónico.
Luego, haz que el SDR responda para continuar la conversación. Esto puede requerir algo de programación de ambiente. 😂
</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
           <h2 style="color:#00bfff;">Implicaciones comerciales</h2>
<span style="color:#00bfff;">Esto se aplica inmediatamente a la automatización de ventas; pero, de forma más general, podría aplicarse a la automatización integral de cualquier proceso empresarial mediante conversaciones y herramientas. Piensa en cómo podrías aplicar una solución de agente como esta en tu trabajo diario.
</span>
        </td>
    </tr>
</table>

## Nota adicional:

Google ha anunciado su Kit de Desarrollo de Agentes (ADK), que se encuentra en una versión preliminar. Aún está en desarrollo, por lo que es demasiado pronto para que podamos hablar de él aquí. Sin embargo, es interesante observar que se parece bastante al SDK de OpenAI Agents. Para que lo veas, aquí tienes un vistazo al código de ejemplo del ADK:


```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agente para responder preguntas sobre la hora y el clima en una ciudad.",
    instruction="Eres un agente útil que puede responder las preguntas de los usuarios sobre la hora y el clima de una ciudad.",
    tools=[get_weather, get_current_time]
)
```

¡Vaya, eso me suena!
